### Ежедневные отбивки

**Отбивки на каждый день**

    Выручка за текущий календарный мес (N+продления) + за сегодня
    Выручка за текущий календарный мес с N + за сегодня
    Выручка за текущий календарный мес с продлений + за сегодня

    Кол-во N оплат за текущий календарный мес  + за сегодня
    Кол-во оплат продлений за текущий календарный мес  + за сегодня

    Средний чек накопительно по N
    Средний чек накопительно по продлениям
    Средний чек накопительно номинальный
    Средний чек накопительно с потенциалом рр

    в конце каждого месяца пометка плана сюда :

    План общий и % выполнения
    План с новых и % выполнения
    План с продлений и % выполнения

    Чтобы заработать (план общий) ...... остаток от плана делю на остаток дней
    Чтобы заработать (план первых оплат) ......
    Чтобы заработать (план продлений оплат) ......

    ПЗ сегодня состоялось
    ПЗ запланировано на завтра
    Конверсия ПЗ - оплата за этот день

In [22]:
from sqlalchemy import create_engine
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import datetime as dt


from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError


### Выгружаю все данные

In [23]:
### Создадим движок для postgresql
engine = create_engine('postgresql://hidden')

In [24]:
### Выгрузим данные из postgresql
query = "SELECT DISTINCT start_time, test_name, id, ended, duration_min, client_opozdal \
               FROM level_1_raw_data.lessons \
               WHERE test_name IS NOT NULL"
pz = pd.read_sql_query(query, engine)



query = "SELECT *  \
         FROM level_2_analytical_tables.transactions \
         WHERE amount is not null"
transactions = pd.read_sql_query(query, engine)

### Расчет пунктов для ежедневной отбивки

**Дата**

In [25]:

# ------------------------------------------------------------------------------------------------------------------------------

# ------------------------------------------------------для расчета ежедневной отбивки------------------------------------------

# Получение вчерашней даты
current_date = pd.Timestamp.today().date() - timedelta(days=1)
# current_date = pd.Timestamp.today().date() - timedelta(days=1)

# Получение следующего дня от current_date
tommorow = current_date + timedelta(days=1)

end_of_month = current_date + relativedelta(day=31)
months_remaining = (end_of_month - current_date)
months_remaining = months_remaining.days


# ------------------------------------------------------для расчета еженедельной отбивки----------------------------------------

# Получаем текущую дату
today = dt.date.today()
yesterday = today - dt.timedelta(days=1)

# Определяем текущий день недели (0 - понедельник, 6 - воскресенье)
weekday = today.weekday()

#-------------------------------------------------- Неделя

# Дата прошлого понедельника
start_week = today - dt.timedelta(days=7)

# Дата прошлого воскресенья
end_week = today - dt.timedelta(days=1)

# Даты для теста 
# start_week = today - dt.timedelta(days=7)
# end_week = today - dt.timedelta(days=1)


# -------------------------------------------------- Месяц

# Получение текущего календарного месяца и следующего
current_month = current_date.replace(day=1)

next_month = current_month + relativedelta(months=1)
# end_of_month = end_week + relativedelta(day=31)



# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------для расчета ежемесячной отбивки----------------------------------------

last_day_of_month = current_month + pd.offsets.MonthEnd(0)
end_of_month = last_day_of_month.date()



# March 

plan = 30810000 #money 
plan_1N = 15810000
plan_0 = 15000000

N_avg_plan= 31000 #sr.check
O_avg_plan = 25000

count_1N_plan = 510 #kol-vo prodazh
count_0_plan = 600

count_plan = 1110


# Подбивать 1 числа
lidov_plan_month = 6000 #lidov medium plan
returns_month = 0


# Подбивать по понедельникам
returns = 116044


foreign_payment = 0

In [ ]:

def calculate_everyday_data(transactions, pz, current_date, current_month, next_month, tommorow,\
                    end_of_month, months_remaining, plan,plan_1N, plan_0, N_avg_plan, count_1N_plan, count_0_plan, count_plan):


    transactions['amount'] = transactions['amount'].astype('float')
    pz['start_time'] = pd.to_datetime(pz['start_time'])

    # Выручка за текущий календарный мес
    sum_for_current_day = transactions[transactions['date'].dt.date == current_date]['amount'].sum()
    sum_for_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month)]['amount'].sum()


    # Выручка за текущий календарный мес с N + за сегодня
    N_sum_current_day = transactions[(transactions['date'].dt.date == current_date) & (transactions['first_sale'] == 1)]['amount'].sum()
    N_sum_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['first_sale'] == 1)]['amount'].sum()

    # Выручка за текущий календарный мес с продлений + за сегодня
    sec_sum_current_day = transactions[(transactions['date'].dt.date == current_date) & (transactions['first_sale'] == 0)]['amount'].sum()
    sec_sum_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['first_sale'] == 0)]['amount'].sum()

    
    
    
    
    # Кол-во всех продаж 
    сount_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['rn_package_id'] == 1)]['package_id'].nunique()

    count_current_day = transactions[(transactions['date'].dt.date == current_date) & (transactions['rn_package_id'] == 1)]['package_id'].nunique()


    
    # Первичные
    # Выбор чисел за текущий день и подсчет их кол-ва - по первичным продажам
    N_count_current_day = transactions[(transactions['date'].dt.date == current_date) & (transactions['first_sale'] == 1) & (transactions['rn_package_id'] == 1)]['package_id'].nunique()


    # Выбор чисел за текущий календарный месяц и подсчет их кол-ва - по первичным продажам
    N_count_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['first_sale'] == 1) & (transactions['rn_package_id'] == 1)]['package_id'].nunique()


    
    # Вторичные
    # Выбор чисел за текущий день и подсчет их кол-ва - по вторичным продажам
    second_count_current_day = transactions[(transactions['date'].dt.date == current_date) & (transactions['first_sale'] == 0) & (transactions['rn_package_id'] == 1)]['package_id'].nunique()


    # Выбор чисел за текущий календарный месяц и подсчет их кол-ва - по вторичным продажам
    second_count_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['first_sale'] == 0) & (transactions['rn_package_id'] == 1)]['package_id'].nunique()



    
    
    # Средний чек накопительно по N

    # Первичные
    # Выбор чисел за текущий календарный месяц и подсчет их ср. чека - по первичным продажам
    N_avg_first_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['first_sale'] == 1) & (transactions['rn_package_id'] == 1)]['amount'].mean()


    # Средний чек накопительно по продлениям
    # Выбор чисел за текущий календарный месяц и подсчет их ср. чека - по вторичным продажам
    avg_second_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['date'].dt.date < next_month) & (transactions['first_sale'] == 0) & (transactions['rn_package_id'] == 1)]['amount'].mean()

    

    # Средний чек накопительно номинальный
    avg_current_month = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['rn_package_id'] == 1)]['amount'].mean()


    # Средний чек накопительно с потенциалом рр
    avg_current_month_rr = transactions[(transactions['date'].dt.date >= current_month) & (transactions['date'].dt.date < next_month) & (transactions['rn_package_id'] == 1)]['amount_full'].mean()

    
    

    # ПЗ вчера
    pz_today = pz[pz['start_time'].dt.date == current_date]
    pz_today = pz_today[(pz_today['ended']== True) & (pz_today['duration_min'] > 15) & (pz_today['client_opozdal'] != True)]['id'].nunique()

    
    # ПЗ запланировано на завтра
    pz_tommorow = pz[pz['start_time'].dt.date == tommorow]
    pz_tommorow = pz_tommorow['id'].nunique()

    
    
    
    # Формирование сообщения
    
    if  sum_for_current_day > 2000000 : 
        
        message = f"Пчелка в пути..."
        
    else :
        message = (

            f"Данные на *{current_date}* (вчера)\n"
            "\n"
            "\n"
            f"Выручка с начала мес  {sum_for_current_month:,.0f}, за вчера {sum_for_current_day:,.0f}\n"
            f"с новых {N_sum_current_month:,.0f}, за вчера {N_sum_current_day:,.0f}\n"
            f"с продлений {sec_sum_current_month:,.0f}, за вчера {sec_sum_current_day:,.0f}\n"
            "\n"
            f"Кол-во оплат с начала мес {сount_current_month:,.0f}, за вчера {count_current_day:,.0f}\n"
            f"новых {N_count_current_month:,.0f}, за вчера {N_count_current_day:,.0f}\n"
            f"с продлений {second_count_current_month:,.0f}, за вчера {second_count_current_day:,.0f}\n"
            "\n"
            f"Ср. чек с начала мес фактический : {round(avg_current_month):,.0f} рублей\n"
            f"Ср. чек с начала мес с потенциалом рр : {round(avg_current_month_rr):,.0f} рублей\n"
             "\n"
            f"Ср. чек с начала мес по новым : {round(N_avg_first_current_month):,.0f} рублей\n"
            f"Ср. чек с начала мес по продлениям : {round(avg_second_current_month):,.0f} рублей\n"
            "\n"
            f"ПЗ вчера {pz_today} , ПЗ запланировано на сегодня {pz_tommorow}\n"
            "\n"
            "\n"
            f"*План:*"
            "\n"
            "\n"
            f"Выручка {plan:,.0f}₽ (выполнен на {round((sum_for_current_month/ plan) * 100)})%\n"
            f" с новых: {plan_1N:,.0f}₽ (выполнен на {round((N_sum_current_month/ plan_1N) * 100)})%\n"
            f" с продлений: {plan_0:,.0f}₽ (выполнен на {round((sec_sum_current_month/ plan_0) * 100)})%\n"
            "\n"
            f"План среднего чека с новых {N_avg_plan:,.0f}₽ (выполнен на {round((N_avg_first_current_month / N_avg_plan) * 100)})%\n"
            f"План среднего чека с продлений {O_avg_plan:,.0f}₽ (выполнен на {round((avg_second_current_month / O_avg_plan) * 100)})%\n"
            "\n"
            f"План кол-ва оплат с новых: {count_1N_plan:,.0f} (выполнен на {round((N_count_current_month/ count_1N_plan) * 100)})%\n"
            f"План кол-ва оплат с продлений: {count_0_plan:,.0f} (выполнен на {round((second_count_current_month/ count_0_plan) * 100)})%\n"
            "\n"
             )


        # Проверяем условие и добавляем соответствующее сообщение
        if months_remaining == 0:
            message += "Месяц подошел к концу\n"
        else:
            message += (
                f"Чтобы выполнить план по выручке нужно {round((plan - sum_for_current_month) / months_remaining):,.0f}₽ и {round((count_plan - сount_current_month) / months_remaining):,.0f} оплат в день до конца месяца\n"
                f"по новым {round((plan_1N - N_sum_current_month) / months_remaining):,.0f}₽ и {round((count_1N_plan - N_count_current_month) / months_remaining):,.0f} оплат в день до конца месяца\n"
                f"по продлениям {round((plan_0 - sec_sum_current_month) / months_remaining):,.0f}₽ и {round((count_0_plan - second_count_current_month) / months_remaining):,.0f} оплат в день до конца месяца\n"
            )
        
    print(sum_for_current_day)

    
    return message




In [30]:
message = calculate_everyday_data(transactions = transactions,pz= pz ,
                                  next_month= next_month,  current_date=current_date, 
                                  current_month=current_month, tommorow=tommorow, end_of_month=end_of_month, 
                                  months_remaining=months_remaining, plan=plan, plan_1N=plan_1N, plan_0=plan_0 , 
                                  N_avg_plan = N_avg_plan, count_1N_plan= count_1N_plan, count_0_plan=count_0_plan, 
                                  count_plan=count_plan)

88440.0


In [31]:
print(message)

Данные на *2024-05-01* (вчера)


Выручка с начала мес  88,440, за вчера 88,440
с продлений 88,440, за вчера 88,440

Кол-во оплат с начала мес 6, за вчера 6
с продлений 6, за вчера 6

Ср. чек с начала мес фактический : 14,740 рублей
Ср. чек с начала мес с потенциалом рр : 14,740 рублей

Ср. чек с начала мес по продлениям : 14,740 рублей



*План:*

Выручка 30,810,000₽ (выполнен на 0)%
 с продлений: 15,000,000₽ (выполнен на 1)%

План среднего чека с продлений 25,000₽ (выполнен на 59)%

План кол-ва оплат с продлений: 600 (выполнен на 1)%

Чтобы выполнить план по выручке нужно 1,024,052₽ и 37 оплат в день до конца месяца
по продлениям 497,052₽ и 20 оплат в день до конца месяца


### Загрузка в slack


In [8]:

def load_usedesk_tickets(message):
    client = WebClient(token="hidden")

    try:
        response = client.chat_postMessage(
#                          channel ="#test_otbivok",
                        channel="#product-dashboard",
            text=message

        )

        print("Сообщение отправлено успешно!")
    except SlackApiError as e:
        print("Ошибка отправки сообщения:", e)


In [9]:
load_usedesk_tickets(message=message)

Сообщение отправлено успешно!
